In [1]:
import numpy as np
import matplotlib.pyplot as plt

## grille sauts

In [2]:
def grille_sauts(T, kn,lamda):
    jumps_t = []
    grid = []
    x =  np.random.exponential(1/lamda)
    sum_x = x
    
    vec = []
    t = 0
    vec.append(t)
    vec.append(0)
    grid.append(vec)
    vec = []
    
    # construction of the jump times
    while sum_x < T:
        jumps_t.append(sum_x)
        x = np.random.exponential(1/lamda)
        sum_x += x
    
    i = 0
    # construction of the grid on the interval [0,last jump]
    jump_size = len(jumps_t)
    epsilon = 0
    while t < T and i < jump_size:
        epsilon = 0
        t = min(t + 1.0/kn, jumps_t[i])
        if t == jumps_t[i]:
            i += 1
            epsilon = 1 # it is a jump time
        
        vec.append(t)
        vec.append(epsilon)
        grid.append(vec)
        
        vec = []
    
    # to construct the grid on the inverval ]last jump, T]
    while t < T:
        t += 1.0/kn
        
        vec.append(t)
        vec.append(0) # epsilon = 0
        grid.append(vec)
        
        vec = []
    
    vec = []
    vec.append(T)
    vec.append(0)
    
    if grid[-1][0] < T:
        grid.append(vec)
    
    if grid[-1][0] > T:
        grid.pop()
        grid.append(vec)
    
    return grid


## call

In [3]:
import math
import numpy as np

def inverted_h_kou(u, lambda_moins, lambda_plus, p):
    if u < 1 - p:
        return (u / (1 - p))**(1 / lambda_moins) - 1
    else:
        return (p / (1 - u))**(1 / lambda_plus) - 1

def kou_euler_scheme_call(M, strike, x0, T, kn, c, r, lambda_, lambda_plus, lambda_moins, p):
    result = []
    interm=[]
    x = x0
    b =r -c/2- lambda_*(p*(lambda_plus/(lambda_plus - 1)) + (1-p)*lambda_moins/(lambda_moins+1) -1 )
    for j in range(1, M):
        x = x0
        grid=grille_sauts(T, kn,lambda_)
        for i in range(1, len(grid)):
            z=np.random.normal(0, 1)
            u=np.random.uniform(0,1)                                           
            h = grid[i][0] - grid[i-1][0]
            x = x * np.exp(b * h + z * np.sqrt(c * h))
            if grid[i][1] != 0:
                saut = inverted_h_kou(u, lambda_moins, lambda_plus, p)
                x = x * (1 + saut)
        interm.append(max(0, x - strike))
    #result.append(np.mean(interm))
    #result.append(1.96*np.std(interm)/np.sqrt(M))
    result = { 'mean': np.mean(interm), 'std': np.std(interm), 
               'lower': np.mean(interm) - 1.96*np.std(interm)/np.sqrt(M), 
               'upper': np.mean(interm) + 1.96*np.std(interm)/np.sqrt(M) }
    return result

def kou_doleans_dade_call(M, p, lambda_, lambda_plus, lambda_moins, strike, x0, T, c, r):
    result = []
    interm=[]
    y = x0
    b = r-c/2- lambda_*(p*(lambda_plus/(lambda_plus - 1)) + (1-p)*lambda_moins/(lambda_moins+1) -1 )
    sqrtc = np.sqrt(c)
    for j in range(1, M):
        saut = 0
        y = x0
        nbre_de_saut = np.random.poisson(lambda_)
        
        for i in range(nbre_de_saut):
            
            u=np.random.uniform(0,1)
            saut = inverted_h_kou(u, lambda_moins, lambda_plus, p)
            y = y * (1+saut)
        z=np.random.normal(0, 1)
        interm.append(max(0,y * np.exp(b + sqrtc*z)-strike ))
    #result.append(np.mean(interm))
    #result.append(1.96*np.std(interm)/np.sqrt(M))
    result = { 'mean': np.mean(interm), 'std': np.std(interm), 
               'lower': np.mean(interm) - 1.96*np.std(interm)/np.sqrt(M), 
               'upper': np.mean(interm) + 1.96*np.std(interm)/np.sqrt(M) }
    return result
def kou_doleans_dade_2_call(M, p, lamda, lamda_plus, lamda_minus, strike, x0, T, c, r):
    result = []
    interm=[]
    b =r -c/2- lambda_*(p*(lambda_plus/(lambda_plus - 1)) + (1-p)*lambda_moins/(lambda_moins+1) -1 )
    sqrtc = np.sqrt(c)
    for j in range(1, M):
        saut = 0
        nombre_de_saut = np.random.poisson(lamda)
        for i in range( int(nombre_de_saut)):
            u=np.random.uniform(0,1)
            H = np.random.binomial(n=1, p=p)
            saut +=(H *(-math.log(u)/lamda_plus) + (1-H)* math.log(u)/lamda_minus)
        z=np.random.normal(0, 1)
        interm.append(max(0, x0 * math.exp(b + sqrtc * z +saut) - strike))
    #result.append(np.mean(interm))
    #result.append(1.96*np.std(interm)/np.sqrt(M))
    result = { 'mean': np.mean(interm), 'std': np.std(interm), 
               'lower': np.mean(interm) - 1.96*np.std(interm)/np.sqrt(M), 
               'upper': np.mean(interm) + 1.96*np.std(interm)/np.sqrt(M) }
    return result

def merton_call(M, lamda, delta, strike, x0, T, r, c):
    result = []
    interm=[]
    #float_M = float(M)
    b = r - c/2 + lamda * (1 - math.exp((delta**2)/2))
    sqrtc = math.sqrt(c)
    for j in range(M):
        z=np.random.normal(0, 1)
        N_T=np.random.poisson(lamda)
        z2=np.random.normal(0, 1,N_T)
        interm.append(max(0, x0*math.exp(b + sqrtc*z + delta*np.sum(z2))- strike))
    #result.append(np.mean(interm))
    #result.append(1.96*np.std(interm)/np.sqrt(M))
    result = { 'mean': np.mean(interm), 'std': np.std(interm), 
               'lower': np.mean(interm) - 1.96*np.std(interm)/np.sqrt(M), 
               'upper': np.mean(interm) + 1.96*np.std(interm)/np.sqrt(M) }
    return result



In [4]:
c = 0.04
lambda_moins=6
lambda_=4
lambda_plus=3
p=0.5
x0=100
strike=[80,100,120]
T=1
kn=1000
M=10000
r=0.02
delta=0.2
m=0

In [5]:
for s in strike:
    print('strike:',s)
    print(kou_euler_scheme_call(M, s, x0, T, kn, c, r, lambda_, lambda_plus, lambda_moins, p))

strike: 80
{'mean': 45.619489524455, 'std': 204.8993327632459, 'lower': 41.60346260229538, 'upper': 49.635516446614616}
strike: 100
{'mean': 37.39415714655488, 'std': 198.6544403633697, 'lower': 33.50053011543284, 'upper': 41.287784177676926}
strike: 120
{'mean': 35.32787579037485, 'std': 171.95587527258962, 'lower': 31.95754063503209, 'upper': 38.698210945717605}


In [6]:
for s in strike:
    print('strike:',s)
    print(kou_doleans_dade_call(M, p, lambda_, lambda_plus, lambda_moins, s, x0, T, c, r))

strike: 80
{'mean': 41.88469302583836, 'std': 147.26793140402413, 'lower': 38.998241570319486, 'upper': 44.77114448135723}
strike: 100
{'mean': 38.62265850833173, 'std': 170.38486634410674, 'lower': 35.28311512798724, 'upper': 41.962201888676226}
strike: 120
{'mean': 33.62809917455475, 'std': 238.35253566707127, 'lower': 28.95638947548015, 'upper': 38.299808873629345}


In [7]:
for s in strike:
    print('strike:',s)
    print(kou_doleans_dade_2_call(M, p, lambda_, lambda_plus, lambda_moins, s, x0, T, c, r))

strike: 80
{'mean': 42.53902288786276, 'std': 163.90662345202992, 'lower': 39.32645306820297, 'upper': 45.75159270752254}
strike: 100
{'mean': 37.96344733434406, 'std': 213.99713881854063, 'lower': 33.769103413500666, 'upper': 42.157791255187455}
strike: 120
{'mean': 33.88081874744275, 'std': 195.40853568137695, 'lower': 30.050811448087764, 'upper': 37.71082604679774}


In [8]:
for s in strike:
    print('strike:',s)
    print(merton_call(M, lambda_, delta, s, x0, T, r, c))

strike: 80
{'mean': 29.159709547596634, 'std': 42.78305825363874, 'lower': 28.321161605825314, 'upper': 29.998257489367955}
strike: 100
{'mean': 18.83575386373814, 'std': 39.008243354807774, 'lower': 18.07119229398391, 'upper': 19.600315433492373}
strike: 120
{'mean': 11.630592707151731, 'std': 32.83446584130631, 'lower': 10.987037176662128, 'upper': 12.274148237641334}


## log-contrat

In [9]:
def inverted_h_kou(u, lambda_moins, lambda_plus, p):
    if u < 1 - p:
        return (u / (1 - p))**(1 / lambda_moins) - 1
    else:
        return (p / (1 - u))**(1 / lambda_plus) - 1

def kou_euler_scheme_log_contrat(M, strike, x0, T, kn, c, r, lambda_, lambda_plus, lambda_moins, p):
    result = []
    interm=[]
    x = x0
    b =r -c/2- lambda_*(p*(lambda_plus/(lambda_plus - 1)) + (1-p)*lambda_moins/(lambda_moins+1) -1 )
    for j in range(1, M):
        
        h = 0
        saut = 0
        x = x0
        #grid[:, 0] = np.linspace(0, T, kn+1)
        
        #grid[1:, 1] = np.random.exponential(lambda_,kn)
        grid=grille_sauts(T, kn,lambda_)
        for i in range(1, len(grid)):
            z=np.random.normal(0, 1)
            u=np.random.uniform(0,1)
            h = grid[i][0] - grid[i-1][0]
            x = x * np.exp(b * h + z * np.sqrt(c * h))
            if grid[i][1] != 0:
                saut = inverted_h_kou(u, lambda_moins, lambda_plus, p)
                x = x * (1 + saut)
        interm.append(-2*np.log( x /strike))
    #result.append(np.mean(interm))
    #result.append(1.96*np.std(interm)/np.sqrt(M))
    result = { 'mean': np.mean(interm), 'std': np.std(interm), 
               'lower': np.mean(interm) - 1.96*np.std(interm)/np.sqrt(M), 
               'upper': np.mean(interm) + 1.96*np.std(interm)/np.sqrt(M) }
    return result

def kou_doleans_dade_log_contrat(M, p, lambda_, lambda_plus, lambda_moins, strike, x0, T, c, r):
    result = []
    interm=[]
    y = x0
    b = r-c/2- lambda_*(p*(lambda_plus/(lambda_plus - 1)) + (1-p)*lambda_moins/(lambda_moins+1) -1 )
    sqrtc = np.sqrt(c)
    for j in range(1, M):
        saut = 0
        y = x0
        nbre_de_saut = np.random.poisson(lambda_)
        
        for i in range(nbre_de_saut):
            
            u=np.random.uniform(0,1)
            saut = inverted_h_kou(u, lambda_moins, lambda_plus, p)
            y = y * (1 + saut)
        z=np.random.normal(0, 1)
        interm.append(-2*np.log( y * np.exp(b + sqrtc*z) /strike))
    #result.append(np.mean(interm))
    #result.append(1.96*np.std(interm)/np.sqrt(M))
    result = { 'mean': np.mean(interm), 'std': np.std(interm), 
               'lower': np.mean(interm) - 1.96*np.std(interm)/np.sqrt(M), 
               'upper': np.mean(interm) + 1.96*np.std(interm)/np.sqrt(M) }
    return result
        

def kou_doleans_dade_bis_log_contrat(M, p, lamda, lamda_plus, lamda_minus, strike, x0, T, c, r):
    result = []
    interm=[]
    b =r -c/2- lambda_*(p*(lambda_plus/(lambda_plus - 1)) + (1-p)*lambda_moins/(lambda_moins+1) -1 )
    sqrtc = math.sqrt(c)
    for j in range(1, M):
        saut = 0
        nombre_de_saut = np.random.poisson(lamda)
        for i in range(1, int(nombre_de_saut)):
            u=np.random.uniform(0,1)
            H = np.random.binomial(n=1, p=p)
            saut +=( H *(-math.log(u)/lamda_plus) + (1-H)* math.log(u)/lamda_minus)
        z=np.random.normal(0, 1)
        
        interm.append(-2*np.log(x0 * math.exp(b + sqrtc * z +saut) / strike))
        
    #result.append(np.mean(interm))
    #result.append(1.96*np.std(interm)/np.sqrt(M))
    result = { 'mean': np.mean(interm), 'std': np.std(interm), 
               'lower': np.mean(interm) - 1.96*np.std(interm)/np.sqrt(M), 
               'upper': np.mean(interm) + 1.96*np.std(interm)/np.sqrt(M) }
    return result


def merton_log_contrat(M, lamda, delta, strike, x0, T, r, c):
    result = []
    interm=[]
    b = r - c/2 + lamda * (1 - math.exp((delta**2)/2))
    sqrtc = math.sqrt(c)
    for j in range(M):
        z=np.random.normal(0, 1)
        N_T=np.random.poisson(lamda)
        z2=np.random.normal(0, 1)
       
        interm.append( -2*np.log(x0 * math.exp(b + sqrtc*z + delta*math.sqrt(N_T)*z2) /strike))
    #result.append(np.mean(interm))
    #result.append(1.96*np.std(interm)/np.sqrt(M))
    result = { 'mean': np.mean(interm), 'std': np.std(interm), 
               'lower': np.mean(interm) - 1.96*np.std(interm)/np.sqrt(M), 
               'upper': np.mean(interm) + 1.96*np.std(interm)/np.sqrt(M) }
    return result



In [10]:
b = 0.01
c = 0.04
lambda_moins=6
lambda_=4
lambda_plus=3
p=0.5
x0=100
strike=[80,100,120]
T=1
kn=1000
M=10000
r=0.02
delta=0.2


In [11]:
for s in strike:
    print('strike:',s)
    print(kou_euler_scheme_log_contrat(M, s, x0, T, kn, c, r, lambda_, lambda_plus, lambda_moins, p))

strike: 80
{'mean': 0.3119278551986322, 'std': 1.5542781457828532, 'lower': 0.28146400354128825, 'upper': 0.34239170685597614}
strike: 100
{'mean': 0.7557399262586325, 'std': 1.5448014438442514, 'lower': 0.7254618179592852, 'upper': 0.7860180345579798}
strike: 120
{'mean': 1.11361547283674, 'std': 1.5616380105046, 'lower': 1.0830073678308498, 'upper': 1.1442235778426302}


In [12]:
for s in strike:
    print('strike:',s)
    print(kou_doleans_dade_log_contrat(M, p, lambda_, lambda_plus, lambda_moins, s, x0, T, c, r))

strike: 80
{'mean': 0.32305415207132304, 'std': 1.5553396410448892, 'lower': 0.2925694951068432, 'upper': 0.35353880903580287}
strike: 100
{'mean': 0.7616597273470848, 'std': 1.5307401071022437, 'lower': 0.7316572212478808, 'upper': 0.7916622334462887}
strike: 120
{'mean': 1.1409919736748855, 'std': 1.5408460069140615, 'lower': 1.11079139193937, 'upper': 1.171192555410401}


In [13]:
for s in strike:
    print('strike:',s)
    print(kou_doleans_dade_2_log_contrat(M, p, lambda_, lambda_plus, lambda_moins, s, x0, T, c, r))

strike: 80


NameError: name 'kou_doleans_dade_2_log_contrat' is not defined

In [ ]:
for s in strike:
    print('strike:',s)
    print(merton_log_contrat(M, lambda_, delta, s, x0, T, r, c))